In [2]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import seaborn as sns
sns.set_style("white")

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = io.loadmat("/media/zuziel/Windows/Users/Zuza/Documents/Programowanie/keras/notMNIST_small.mat")

In [4]:
X = data['images']
y = data['labels']
resolution = 28
classes = 10

X = np.transpose(X, (2, 0, 1))
X = X.reshape(X.shape[0], 1, 28, 28)

y = y.astype('int32')
X = X.astype('float32') / 255.

# 3 -> [0., 0., 0., 2., 0., 0., 0., 0., 0., 0.]
Y = np_utils.to_categorical(y, 10)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

Sieć podobna do poprzedniej jednak ze zwiększonym rozmiarem warstwy - 512 neuronów zamiast 128.

In [5]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(classes, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
from keras_ascii_sequential import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (1, 28, 28)
  Convolution2D    \|/  -------------------       832     0.0%
           relu   #####   (32, 24, 24)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (16, 12, 24)
        Dropout    | || -------------------         0     0.0%
                  #####   (16, 12, 24)
        Flatten   ||||| -------------------         0     0.0%
                  #####   (4608,)
          Dense   XXXXX -------------------   2359808    89.8%
           relu   #####   (512,)
          Dense   XXXXX -------------------    262656    10.0%
           relu   #####   (512,)
          Dense   XXXXX -------------------      5130     0.2%
        softmax   #####   (10,)


In [8]:
# checkpoint
filepath="net3_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 35s - loss: 0.4064 - acc: 0.8780 - val_loss: 0.2950 - val_acc: 0.9170
Epoch 2/10
14979/14979 [==============================] - 33s - loss: 0.2171 - acc: 0.9324 - val_loss: 0.2381 - val_acc: 0.9311
Epoch 3/10
14979/14979 [==============================] - 33s - loss: 0.1504 - acc: 0.9519 - val_loss: 0.2735 - val_acc: 0.9204
Epoch 4/10
13376/14979 [=========================>....] - ETA: 3s - loss: 0.1107 - acc: 0.9623 

In [10]:
print(history.history)

{'val_acc': [0.91695594125500668, 0.93110814419225629, 0.92042723631508683, 0.93164218958611478, 0.9324432576769025, 0.92950600801068095, 0.93591455273698265, 0.93164218958611478, 0.93724966622162886, 0.93457943925233644], 'val_loss': [0.29504936963757461, 0.2380975017719498, 0.27348627697959921, 0.25613518553836961, 0.25610439475720254, 0.28655240149280736, 0.30359857865747247, 0.35960222692471322, 0.29800261910681491, 0.31881111706936549], 'acc': [0.87796248080646233, 0.93243874757994527, 0.95193270578810329, 0.96281460711662992, 0.97309566727148711, 0.97770211629614789, 0.98310968689498635, 0.9869150143534281, 0.98758261566192673, 0.98811669670872559], 'loss': [0.4064180587802434, 0.21707682919690885, 0.15038306884687952, 0.11068425961172434, 0.081083337620116575, 0.068718895423578286, 0.050288891331557242, 0.041434464267260926, 0.042335752387009984, 0.036302734460745535]}


W tym wypadku już po pierwszych 10 iteracjach dochodzimy do wyników podobnych do tych widocznych po 20 iteracji.

In [11]:
score = model.evaluate(X,Y)
print(score)

18720/18724 [============================>.] - ETA: 0s    64/18724 [..............................] - ETA: 14s  480/18724 [..............................] - ETA: 17s

In [12]:
print(model.evaluate(X_test,Y_test))

3744/3745 [============================>.] - ETA: 0s  64/3745 [..............................] - ETA: 2s - ETA: 3s

In [ ]:
history = model.fit(X_train, Y_train,
                    nb_epoch=10,
                    batch_size=32,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list)

Train on 14979 samples, validate on 3745 samples
Epoch 1/10
14979/14979 [==============================] - 35s - loss: 0.0343 - acc: 0.9899 - val_loss: 0.3542 - val_acc: 0.9284
Epoch 2/10
14979/14979 [==============================] - 32s - loss: 0.0387 - acc: 0.9887 - val_loss: 0.3458 - val_acc: 0.9340
Epoch 3/10
14979/14979 [==============================] - 32s - loss: 0.0222 - acc: 0.9931 - val_loss: 0.3393 - val_acc: 0.9386
Epoch 4/10
 8480/14979 [===============>..............] - ETA: 13s - loss: 0.0172 - acc: 0.9952

In [14]:
print(history.history)

{'val_acc': [0.92843791722296398, 0.93404539385847796, 0.93858477970627507, 0.93004005340453944, 0.93484646194926568, 0.93511348464619493, 0.93671562082777038, 0.93030707610146868, 0.93457943925233644, 0.93591455273698265], 'val_loss': [0.35424124326343054, 0.34578669953106522, 0.33929050506822495, 0.39147510699901694, 0.3931870935133811, 0.3673832126600044, 0.39661767179732682, 0.46644544390131076, 0.42020702027211848, 0.39435547959190115], 'acc': [0.98985246011480099, 0.98871753788637429, 0.9930569463916149, 0.99379130783096337, 0.9930569463916149, 0.99352426730756394, 0.99352426730756394, 0.99332398691501433, 0.99526003070966018, 0.99392482809266303], 'loss': [0.034305932602539668, 0.038684533115178041, 0.022222858522936692, 0.019815893457933945, 0.023343495764793932, 0.021846302626314991, 0.021778490149829181, 0.02479995302798248, 0.016867234369373395, 0.019908045838404546]}


Kolejne 10 iteracji niewiele zmieniło. Jeśli chodzi o zbiór testowy najwyższy wynik uzyskano w 6. iteracjiz pierwszego uczenia.

In [16]:
best_model =  Sequential()
best_model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, 28, 28), activation='relu', dim_ordering="th"))
best_model.add(MaxPooling2D(pool_size=(2, 2)))
best_model.add(Dropout(0.2))
best_model.add(Flatten())
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(512, activation='relu'))
best_model.add(Dense(classes, activation='softmax'))
# Compile model
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
best_model.load_weights("net3_weights-improvement-02-0.94.hdf5")

In [17]:
print(best_model.evaluate(X,Y))
print(best_model.evaluate(X_train,Y_train))
print(best_model.evaluate(X_test,Y_test))

3744/3745 [============================>.] - ETA: 0s

# Poszukiwanie najczęściej mylonych liter

In [18]:
predictions = best_model.predict(X)

In [19]:
errors = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
for i in range(0,X.shape[0]):
    errors[np.argmax(Y[i])][np.argmax(predictions[i])] += 1
print(errors)

[[1848, 4, 2, 3, 0, 1, 0, 13, 1, 0], [6, 1849, 2, 2, 4, 0, 6, 3, 1, 0], [0, 1, 1837, 1, 11, 1, 17, 1, 4, 0], [4, 13, 2, 1837, 0, 0, 3, 2, 6, 6], [1, 4, 5, 1, 1855, 0, 4, 0, 3, 0], [3, 4, 1, 2, 9, 1844, 0, 2, 1, 6], [2, 2, 22, 1, 6, 2, 1836, 0, 0, 1], [8, 4, 2, 1, 3, 0, 1, 1848, 3, 2], [6, 2, 3, 5, 0, 2, 1, 3, 1822, 28], [1, 1, 2, 4, 1, 0, 1, 1, 9, 1852]]


Wynika z tego, że najczęściej mylone są:
 1. I z J (28 razy)
 2. G z C (22 razy)
 3. C z G (17 razy)
 4. A z H (13 razy)
  . D z B (13 razy)
 6. C z E (11 razy)

Wyniki te są lepsze niż w przypadku sieci II ale są nieco gorsze od sieci I